# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [283]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Generate the mathematical model

In [284]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [285]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Food and Beverage Production Planning

**Sets:**

*  *T*: Set of time periods (t = 1, 2, ..., 12)
*  *F*: Set of food products (f = 1, 2)
*  *B*: Set of beverage products (b = 1, 2, 3)

**Parameters:**

*  *FC<sub>ft</sub>*: Fixed cost of producing one unit of food *f* in time period *t* (from "fixed_cost_production.csv")
*  *VC<sub>ft</sub>*: Variable cost of producing one unit of food *f* in time period *t* (from "variable_cost_production.csv")
*  *VC<sub>bt</sub>*: Variable cost of producing one liter of beverage *b* in time period *t* (from "variable_cost_production.csv")
*  *SC<sub>ft</sub>*: Variable cost of storing one unit of food *f* in time period *t* (from "variable_cost_storage.csv")
*  *SC<sub>bt</sub>*: Variable cost of storing one liter of beverage *b* in time period *t* (from "variable_cost_storage.csv")
*  *D<sub>ft</sub>*: Demand for food *f* in time period *t* (from "demand.csv")
*  *D<sub>bt</sub>*: Demand for beverage *b* in time period *t* (from "demand.csv")
*  *R<sub>ft</sub>*: Revenue from selling one unit of food *f* in time period *t* (from "revenue.csv")
*  *R<sub>bt</sub>*: Revenue from selling one liter of beverage *b* in time period *t* (from "revenue.csv")
*  *CapS<sub>f</sub>*: Storage capacity for food *f* (580 for f=1, 687 for f=2)
*  *CapS<sub>b</sub>*: Storage capacity for beverage *b* (599 for b=1, 788 for b=2, 294 for b=3)
*  *CapP<sub>f</sub>*: Production capacity for food *f* (1080 for f=1, 908 for f=2)
*  *CapP<sub>b</sub>*: Production capacity for beverage *b* (408 for b=1, 1000 for b=2, 403 for b=3)

**Decision Variables:**

*  *P<sub>ft</sub>*: Quantity of food *f* produced in time period *t* (integer)
*  *P<sub>bt</sub>*: Quantity of beverage *b* produced in time period *t* (continuous)
*  *S<sub>ft</sub>*: Quantity of food *f* stored at the end of time period *t* (integer)
*  *S<sub>bt</sub>*: Quantity of beverage *b* stored at the end of time period *t* (continuous)
*  *Sales<sub>ft</sub>*: Quantity of food *f* sold in time period *t* (integer)
*  *Sales<sub>bt</sub>*: Quantity of beverage *b* sold in time period *t* (continuous)

**Objective Function:**

Maximize total profit over the planning horizon:

```
Maximize ∑_{t∈T} ( ∑_{f∈F} ( R<sub>ft</sub> * Sales<sub>ft</sub> - FC<sub>ft</sub> * (P<sub>ft</sub> > 0) - VC<sub>ft</sub> * P<sub>ft</sub> - SC<sub>ft</sub> * S<sub>ft</sub> ) + ∑_{b∈B} ( R<sub>bt</sub> * Sales<sub>bt</sub> - VC<sub>bt</sub> * P<sub>bt</sub> - SC<sub>bt</sub> * S<sub>bt</sub> ) ) 
```

**Constraints:**

* **Production Capacity:**
    * For all *f ∈ F, t ∈ T*: P<sub>ft</sub> ≤ CapP<sub>f</sub>
    * For all *b ∈ B, t ∈ T*: P<sub>bt</sub> ≤ CapP<sub>b</sub>

* **Storage Capacity:**
    * For all *f ∈ F, t ∈ T*: S<sub>ft</sub> ≤ CapS<sub>f</sub>
    * For all *b ∈ B, t ∈ T*: S<sub>bt</sub> ≤ CapS<sub>b</sub>

* **Inventory Balance:**
    * For all *f ∈ F, t ∈ T*: S<sub>f,t-1</sub> + P<sub>ft</sub> - Sales<sub>ft</sub> = S<sub>ft</sub>  (where S<sub>f,0</sub> = 0)
    * For all *b ∈ B, t ∈ T*: S<sub>b,t-1</sub> + P<sub>bt</sub> - Sales<sub>bt</sub> = S<sub>bt</sub>  (where S<sub>b,0</sub> = 0)

* **Demand Satisfaction (Maximum Sales):**
    * For all *f ∈ F, t ∈ T*: Sales<sub>ft</sub> ≤ D<sub>ft</sub> 
    * For all *b ∈ B, t ∈ T*: Sales<sub>bt</sub> ≤ D<sub>bt</sub>

* **Non-negativity:**
    * For all *f ∈ F, t ∈ T*: P<sub>ft</sub>, S<sub>ft</sub>, Sales<sub>ft</sub> ≥ 0 
    * For all *b ∈ B, t ∈ T*: P<sub>bt</sub>, S<sub>bt</sub>, Sales<sub>bt</sub> ≥ 0

This model defines the decision variables, parameters, objective function, and constraints necessary to determine the optimal production plan for the food and beverage facility over the 12 time periods. The model aims to maximize profit while considering production costs, storage costs, demand limitations, and capacity constraints. 


## 3. Generate the pyomo code

In [286]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [287]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data from CSV files)
fixed_cost_production = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  
variable_cost_production = {(1, 1): 10, (1, 2): 12, (2, 1): 11, (2, 2): 13,
                           (3, 1): 5, (3, 2): 6, (4, 1): 4, (4, 2): 5}
variable_cost_storage = {(1, 1): 2, (1, 2): 2.5, (2, 1): 3, (2, 2): 3.5,
                          (3, 1): 1, (3, 2): 1.2, (4, 1): 0.8, (4, 2): 1}
demand = {(1, 1): 200, (1, 2): 250, (2, 1): 180, (2, 2): 220,
          (3, 1): 150, (3, 2): 170, (4, 1): 120, (4, 2): 140}
revenue = {(1, 1): 30, (1, 2): 32, (2, 1): 28, (2, 2): 30,
           (3, 1): 22, (3, 2): 25, (4, 1): 18, (4, 2): 20} 

# Sets
T = range(1, 13)  # Time periods
F = [1, 2]  # Food products
B = [1, 2, 3]  # Beverage products

# Model
model = pyo.ConcreteModel()

# Parameters
model.FC = pyo.Param(F, T, initialize=fixed_cost_production)
model.VC = pyo.Param(F, T, initialize=variable_cost_production, within=pyo.NonNegativeReals)
model.VC_b = pyo.Param(B, T, initialize=variable_cost_production, within=pyo.NonNegativeReals)
model.SC = pyo.Param(F, T, initialize=variable_cost_storage, within=pyo.NonNegativeReals)
model.SC_b = pyo.Param(B, T, initialize=variable_cost_storage, within=pyo.NonNegativeReals)
model.D = pyo.Param(F, T, initialize=demand, within=pyo.NonNegativeIntegers)
model.D_b = pyo.Param(B, T, initialize=demand, within=pyo.NonNegativeReals)
model.R = pyo.Param(F, T, initialize=revenue, within=pyo.NonNegativeReals)
model.R_b = pyo.Param(B, T, initialize=revenue, within=pyo.NonNegativeReals)
model.CapS = pyo.Param(F, initialize={1: 580, 2: 687}, within=pyo.NonNegativeIntegers)
model.CapS_b = pyo.Param(B, initialize={1: 599, 2: 788, 3: 294}, within=pyo.NonNegativeReals)
model.CapP = pyo.Param(F, initialize={1: 1080, 2: 908}, within=pyo.NonNegativeIntegers)
model.CapP_b = pyo.Param(B, initialize={1: 408, 2: 1000, 3: 403}, within=pyo.NonNegativeReals)

# Decision Variables
model.P = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food production
model.P_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage production
model.S = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food storage
model.S_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage storage
model.Sales = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food sales
model.Sales_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage sales

# Objective Function
def objective_rule(model):
    return sum(model.R[f, t] * model.Sales[f, t] - model.FC[f, t] * (model.P[f, t] >= 1) 
               - model.VC[f, t] * model.P[f, t] - model.SC[f, t] * model.S[f, t] 
               + model.R_b[b, t] * model.Sales_b[b, t] - model.VC_b[b, t] * model.P_b[b, t] 
               - model.SC_b[b, t] * model.S_b[b, t] 
               for f in F for b in B for t in T)

model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# Production Capacity
model.production_capacity = pyo.ConstraintList()
for f in F:
    for t in T:
        model.production_capacity.add(model.P[f, t] <= model.CapP[f])

model.production_capacity_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.production_capacity_b.add(model.P_b[b, t] <= model.CapP_b[b])

# Storage Capacity
model.storage_capacity = pyo.ConstraintList()
for f in F:
    for t in T:
        model.storage_capacity.add(model.S[f, t] <= model.CapS[f])

model.storage_capacity_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.storage_capacity_b.add(model.S_b[b, t] <= model.CapS_b[b])

# Inventory Balance
model.inventory_balance = pyo.ConstraintList()
for f in F:
    for t in T:
        if t == 1:
            model.inventory_balance.add(model.P[f, t] - model.Sales[f, t] == model.S[f, t]) 
        else:
            model.inventory_balance.add(model.S[f, t - 1] + model.P[f, t] - model.Sales[f, t] == model.S[f, t])

model.inventory_balance_b = pyo.ConstraintList()
for b in B:
    for t in T:
        if t == 1:
            model.inventory_balance_b.add(model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]) 
        else:
            model.inventory_balance_b.add(model.S_b[b, t - 1] + model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t])

# Demand Satisfaction
model.demand_satisfaction = pyo.ConstraintList()
for f in F:
    for t in T:
        model.demand_satisfaction.add(model.Sales[f, t] <= model.D[f, t])

model.demand_satisfaction_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.demand_satisfaction_b.add(model.Sales_b[b, t] <= model.D_b[b, t])

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
solver.solve(model) 

# Display Results (optional)
print("Profit:", pyo.value(model.profit))
# Add code to print other relevant information as needed 
```

This Pyomo code represents the mathematical optimization model provided, including the objective function and all the constraints.  Remember that this script uses sample data; you need to replace it with your actual data from the CSV files mentioned in the problem description. 


## 4. Input problem data and try running the generated code

In [295]:
import pyomo.environ as pyo

# Sample Data (replace with actual data from CSV files)
# fixed_cost_production = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  
# variable_cost_production = {(1, 1): 10, (1, 2): 12, (2, 1): 11, (2, 2): 13,
#                            (3, 1): 5, (3, 2): 6, (4, 1): 4, (4, 2): 5}
# variable_cost_storage = {(1, 1): 2, (1, 2): 2.5, (2, 1): 3, (2, 2): 3.5,
#                           (3, 1): 1, (3, 2): 1.2, (4, 1): 0.8, (4, 2): 1}
# demand = {(1, 1): 200, (1, 2): 250, (2, 1): 180, (2, 2): 220,
#           (3, 1): 150, (3, 2): 170, (4, 1): 120, (4, 2): 140}
# revenue = {(1, 1): 30, (1, 2): 32, (2, 1): 28, (2, 2): 30,
#            (3, 1): 22, (3, 2): 25, (4, 1): 18, (4, 2): 20} 

#LOADING DATA BY HUMAN
import pandas as pd

# Read the dataframes
fixed_cost_production = pd.read_csv("fixed_cost_production.csv")
variable_cost_production = pd.read_csv("variable_cost_production.csv")
variable_cost_storage = pd.read_csv("variable_cost_storage.csv")
demand = pd.read_csv("demand.csv")
revenue = pd.read_csv("revenue.csv")

# Transform into dictionary format
fixed_cost_production = {(i, j): fixed_cost_production.iloc[i-1, j-1] for i in range(1, len(fixed_cost_production)+1) for j in range(1, len(fixed_cost_production.columns)+1)}
variable_cost_production = {(i, j): variable_cost_production.iloc[i-1, j-1] for i in range(1, len(variable_cost_production)+1) for j in range(1, len(variable_cost_production.columns)+1)}
variable_cost_storage = {(i, j): variable_cost_storage.iloc[i-1, j-1] for i in range(1, len(variable_cost_storage)+1) for j in range(1, len(variable_cost_storage.columns)+1)}
demand = {(i, j): demand.iloc[i-1, j-1] for i in range(1, len(demand)+1) for j in range(1, len(demand.columns)+1)}
revenue = {(i, j): revenue.iloc[i-1, j-1] for i in range(1, len(revenue)+1) for j in range(1, len(revenue.columns)+1)}

#END

# Sets
T = range(1, 12)  # Time periods
F = [1, 2]  # Food products
B = [1, 2, 3]  # Beverage products

# Model
model = pyo.ConcreteModel()

# Parameters
model.FC = pyo.Param(F, T, initialize=fixed_cost_production)
model.VC = pyo.Param(F, T, initialize=variable_cost_production, within=pyo.NonNegativeReals)
model.VC_b = pyo.Param(B, T, initialize=variable_cost_production, within=pyo.NonNegativeReals)
model.SC = pyo.Param(F, T, initialize=variable_cost_storage, within=pyo.NonNegativeReals)
model.SC_b = pyo.Param(B, T, initialize=variable_cost_storage, within=pyo.NonNegativeReals)
model.D = pyo.Param(F, T, initialize=demand, within=pyo.NonNegativeIntegers)
model.D_b = pyo.Param(B, T, initialize=demand, within=pyo.NonNegativeReals)
model.R = pyo.Param(F, T, initialize=revenue, within=pyo.NonNegativeReals)
model.R_b = pyo.Param(B, T, initialize=revenue, within=pyo.NonNegativeReals)
model.CapS = pyo.Param(F, initialize={1: 580, 2: 687}, within=pyo.NonNegativeIntegers)
model.CapS_b = pyo.Param(B, initialize={1: 599, 2: 788, 3: 294}, within=pyo.NonNegativeReals)
model.CapP = pyo.Param(F, initialize={1: 1080, 2: 908}, within=pyo.NonNegativeIntegers)
model.CapP_b = pyo.Param(B, initialize={1: 408, 2: 1000, 3: 403}, within=pyo.NonNegativeReals)

# Decision Variables
model.P = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food production
model.P_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage production
model.S = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food storage
model.S_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage storage
model.Sales = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food sales
model.Sales_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage sales

# Objective Function
def objective_rule(model):
    return sum(model.R[f, t] * model.Sales[f, t] - model.FC[f, t] * (model.P[f, t] >= 1) 
               - model.VC[f, t] * model.P[f, t] - model.SC[f, t] * model.S[f, t] 
               + model.R_b[b, t] * model.Sales_b[b, t] - model.VC_b[b, t] * model.P_b[b, t] 
               - model.SC_b[b, t] * model.S_b[b, t] 
               for f in F for b in B for t in T)

model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# Production Capacity
model.production_capacity = pyo.ConstraintList()
for f in F:
    for t in T:
        model.production_capacity.add(model.P[f, t] <= model.CapP[f])

model.production_capacity_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.production_capacity_b.add(model.P_b[b, t] <= model.CapP_b[b])

# Storage Capacity
model.storage_capacity = pyo.ConstraintList()
for f in F:
    for t in T:
        model.storage_capacity.add(model.S[f, t] <= model.CapS[f])

model.storage_capacity_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.storage_capacity_b.add(model.S_b[b, t] <= model.CapS_b[b])

# Inventory Balance
model.inventory_balance = pyo.ConstraintList()
for f in F:
    for t in T:
        if t == 1:
            model.inventory_balance.add(model.P[f, t] - model.Sales[f, t] == model.S[f, t]) 
        else:
            model.inventory_balance.add(model.S[f, t - 1] + model.P[f, t] - model.Sales[f, t] == model.S[f, t])

model.inventory_balance_b = pyo.ConstraintList()
for b in B:
    for t in T:
        if t == 1:
            model.inventory_balance_b.add(model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]) 
        else:
            model.inventory_balance_b.add(model.S_b[b, t - 1] + model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t])

# Demand Satisfaction
model.demand_satisfaction = pyo.ConstraintList()
for f in F:
    for t in T:
        model.demand_satisfaction.add(model.Sales[f, t] <= model.D[f, t])

model.demand_satisfaction_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.demand_satisfaction_b.add(model.Sales_b[b, t] <= model.D_b[b, t])

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
solver.solve(model) 

# Display Results (optional)
print("Profit:", pyo.value(model.profit))
# Add code to print other relevant information as needed 

ERROR: Rule failed for Param 'FC' with index (1, 12): KeyError: "Index '(1,
12)' is not valid for indexed component 'FC'"
ERROR: Constructing component 'FC' from data=None failed:
        KeyError: "Index '(1, 12)' is not valid for indexed component 'FC'"


KeyError: "Index '(1, 12)' is not valid for indexed component 'FC'"

## 5. Correct the code to verify model viability (optional)

In [318]:
import pyomo.environ as pyo

# Sample Data (replace with actual data from CSV files)
# fixed_cost_production = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  
# variable_cost_production = {(1, 1): 10, (1, 2): 12, (2, 1): 11, (2, 2): 13,
#                            (3, 1): 5, (3, 2): 6, (4, 1): 4, (4, 2): 5}
# variable_cost_storage = {(1, 1): 2, (1, 2): 2.5, (2, 1): 3, (2, 2): 3.5,
#                           (3, 1): 1, (3, 2): 1.2, (4, 1): 0.8, (4, 2): 1}
# demand = {(1, 1): 200, (1, 2): 250, (2, 1): 180, (2, 2): 220,
#           (3, 1): 150, (3, 2): 170, (4, 1): 120, (4, 2): 140}
# revenue = {(1, 1): 30, (1, 2): 32, (2, 1): 28, (2, 2): 30,
#            (3, 1): 22, (3, 2): 25, (4, 1): 18, (4, 2): 20} 

#LOADING DATA BY HUMAN
import pandas as pd

# Read the dataframes
fixed_cost_production = pd.read_csv("fixed_cost_production.csv")
variable_cost_production = pd.read_csv("variable_cost_production.csv")
variable_cost_storage = pd.read_csv("variable_cost_storage.csv")
demand = pd.read_csv("demand.csv")
revenue = pd.read_csv("revenue.csv")

# Define the number of food and beverage items
num_foods = 2
num_beverages = len(fixed_cost_production) - num_foods

# Transform into dictionary format
fixed_cost_production_foods = {(i, j): fixed_cost_production.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(fixed_cost_production.columns))}
fixed_cost_production_beverages = {(i, j): fixed_cost_production.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(fixed_cost_production.columns))}
variable_cost_production_foods = {(i, j): variable_cost_production.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_production_beverages = {(i, j): variable_cost_production.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_storage_foods = {(i, j): variable_cost_storage.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(variable_cost_storage.columns))}
variable_cost_storage_beverages = {(i, j): variable_cost_storage.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(variable_cost_storage.columns))}
demand_foods = {(i, j): demand.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(demand.columns))}
demand_beverages = {(i, j): demand.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(demand.columns))}
revenue_foods = {(i, j): revenue.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(revenue.columns))}
revenue_beverages = {(i, j): revenue.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(revenue.columns))}

#END

# Sets
T = range(1, 13)  # Time periods
F = [1, 2]  # Food products
B = [1, 2, 3]  # Beverage products

# Model
model = pyo.ConcreteModel()

# Parameters
model.FC = pyo.Param(F, T, initialize=fixed_cost_production_foods)
model.VC = pyo.Param(F, T, initialize=variable_cost_production_foods, within=pyo.NonNegativeReals)
model.VC_b = pyo.Param(B, T, initialize=variable_cost_production_beverages, within=pyo.NonNegativeReals)
model.SC = pyo.Param(F, T, initialize=variable_cost_storage_foods, within=pyo.NonNegativeReals)
model.SC_b = pyo.Param(B, T, initialize=variable_cost_storage_beverages, within=pyo.NonNegativeReals)
model.D = pyo.Param(F, T, initialize=demand_foods)
model.D_b = pyo.Param(B, T, initialize=demand_beverages, within=pyo.NonNegativeReals)
model.R = pyo.Param(F, T, initialize=revenue_foods, within=pyo.NonNegativeReals)
model.R_b = pyo.Param(B, T, initialize=revenue_beverages, within=pyo.NonNegativeReals)
model.CapS = pyo.Param(F, initialize={1: 580, 2: 687}, within=pyo.NonNegativeIntegers)
model.CapS_b = pyo.Param(B, initialize={1: 599, 2: 788, 3: 294}, within=pyo.NonNegativeReals)
model.CapP = pyo.Param(F, initialize={1: 1080, 2: 908}, within=pyo.NonNegativeIntegers)
model.CapP_b = pyo.Param(B, initialize={1: 408, 2: 1000, 3: 403}, within=pyo.NonNegativeReals)

# Decision Variables
model.P = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food production
model.P_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage production
model.S = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food storage
model.S_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage storage
model.Sales = pyo.Var(F, T, within=pyo.NonNegativeIntegers) # Food sales
model.Sales_b = pyo.Var(B, T, within=pyo.NonNegativeReals) # Beverage sales

# Objective Function
def objective_rule(model):
    return sum(model.R[f, t] * model.Sales[f, t] - model.FC[f, t] * (model.P[f, t] >= 1) 
               - model.VC[f, t] * model.P[f, t] - model.SC[f, t] * model.S[f, t] 
               + model.R_b[b, t] * model.Sales_b[b, t] - model.VC_b[b, t] * model.P_b[b, t] 
               - model.SC_b[b, t] * model.S_b[b, t] 
               for f in F for b in B for t in T)

model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# Production Capacity
model.production_capacity = pyo.ConstraintList()
for f in F:
    for t in T:
        model.production_capacity.add(model.P[f, t] <= model.CapP[f])

model.production_capacity_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.production_capacity_b.add(model.P_b[b, t] <= model.CapP_b[b])

# Storage Capacity
model.storage_capacity = pyo.ConstraintList()
for f in F:
    for t in T:
        model.storage_capacity.add(model.S[f, t] <= model.CapS[f])

model.storage_capacity_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.storage_capacity_b.add(model.S_b[b, t] <= model.CapS_b[b])

# Inventory Balance
model.inventory_balance = pyo.ConstraintList()
for f in F:
    for t in T:
        if t == 1:
            model.inventory_balance.add(model.P[f, t] - model.Sales[f, t] == model.S[f, t]) 
        else:
            model.inventory_balance.add(model.S[f, t - 1] + model.P[f, t] - model.Sales[f, t] == model.S[f, t])

model.inventory_balance_b = pyo.ConstraintList()
for b in B:
    for t in T:
        if t == 1:
            model.inventory_balance_b.add(model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t]) 
        else:
            model.inventory_balance_b.add(model.S_b[b, t - 1] + model.P_b[b, t] - model.Sales_b[b, t] == model.S_b[b, t])

# Demand Satisfaction
model.demand_satisfaction = pyo.ConstraintList()
for f in F:
    for t in T:
        model.demand_satisfaction.add(model.Sales[f, t] <= model.D[f, t])

model.demand_satisfaction_b = pyo.ConstraintList()
for b in B:
    for t in T:
        model.demand_satisfaction_b.add(model.Sales_b[b, t] <= model.D_b[b, t])

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
solver.solve(model) 

# Display Results (optional)
print("Profit:", pyo.value(model.profit))
# Add code to print other relevant information as needed 

ERROR: Rule failed when generating expression for Objective profit with index
None: TypeError: unsupported operand type(s) for *: 'float' and
'InequalityExpression'
ERROR: Constructing component 'profit' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'float' and
        'InequalityExpression'


TypeError: unsupported operand type(s) for *: 'float' and 'InequalityExpression'

In [309]:
import pandas as pd

# Read the dataframes
fixed_cost_production = pd.read_csv("fixed_cost_production.csv")
variable_cost_production = pd.read_csv("variable_cost_production.csv")
variable_cost_storage = pd.read_csv("variable_cost_storage.csv")
demand = pd.read_csv("demand.csv")
revenue = pd.read_csv("revenue.csv")

# Define the number of food and beverage items
num_foods = 2
num_beverages = len(fixed_cost_production) - num_foods

# Transform into dictionary format
fixed_cost_production_foods = {(i, j): fixed_cost_production.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(fixed_cost_production.columns))}
fixed_cost_production_beverages = {(i, j): fixed_cost_production.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(fixed_cost_production.columns))}
variable_cost_production_foods = {(i, j): variable_cost_production.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_production_beverages = {(i, j): variable_cost_production.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_storage_foods = {(i, j): variable_cost_storage.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(variable_cost_storage.columns))}
variable_cost_storage_beverages = {(i, j): variable_cost_storage.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(variable_cost_storage.columns))}
demand_foods = {(i, j): demand.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(demand.columns))}
demand_beverages = {(i, j): demand.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(demand.columns))}
revenue_foods = {(i, j): revenue.iloc[i-1, j] for i in range(1, num_foods + 1) for j in range(1, len(revenue.columns))}
revenue_beverages = {(i, j): revenue.iloc[i-1+num_foods, j] for i in range(1, num_beverages + 1) for j in range(1, len(revenue.columns))}

# Print the transformed dictionaries
print("fixed_cost_production_foods =", fixed_cost_production_foods)
print("fixed_cost_production_beverages =", fixed_cost_production_beverages)
print("variable_cost_production_foods =", variable_cost_production_foods)
print("variable_cost_production_beverages =", variable_cost_production_beverages)
print("variable_cost_storage_foods =", variable_cost_storage_foods)
print("variable_cost_storage_beverages =", variable_cost_storage_beverages)
print("demand_foods =", demand_foods)
print("demand_beverages =", demand_beverages)
print("revenue_foods =", revenue_foods)
print("revenue_beverages =", revenue_beverages)


fixed_cost_production_foods = {(1, 1): 554.19, (1, 2): 572.0, (1, 3): 529.18, (1, 4): 617.31, (1, 5): 533.74, (1, 6): 543.39, (1, 7): 559.71, (1, 8): 571.31, (1, 9): 595.68, (1, 10): 589.13, (1, 11): 540.62, (1, 12): 606.22, (2, 1): 624.9, (2, 2): 562.7, (2, 3): 310.2, (2, 4): 364.3, (2, 5): 537.9, (2, 6): 358.44, (2, 7): 317.26, (2, 8): 368.13, (2, 9): 410.52, (2, 10): 454.55, (2, 11): 463.93, (2, 12): 429.31}
fixed_cost_production_beverages = {(1, 1): 114.73, (1, 2): 134.42, (1, 3): 128.27, (1, 4): 118.92, (1, 5): 119.16, (1, 6): 125.07, (1, 7): 115.96, (1, 8): 135.59, (1, 9): 134.14, (1, 10): 125.23, (1, 11): 142.87, (1, 12): 134.33, (2, 1): 371.74, (2, 2): 360.78, (2, 3): 401.41, (2, 4): 355.35, (2, 5): 301.21, (2, 6): 350.6, (2, 7): 342.28, (2, 8): 284.81, (2, 9): 338.89, (2, 10): 378.14, (2, 11): 271.69, (2, 12): 350.92, (3, 1): 258.28, (3, 2): 433.32, (3, 3): 573.87, (3, 4): 284.12, (3, 5): 390.95, (3, 6): 184.32, (3, 7): 162.6, (3, 8): -28.18, (3, 9): 362.46, (3, 10): 248.21, (

In [316]:
model.FC()

TypeError: Cannot compute the value of an indexed Param (FC)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [281]:
print(response.text)

## Mathematical Optimization Model for Drug Production

**Sets and Indices:**

*  $i \in \{1,2\}$: Index for raw materials.
*  $j \in \{1,2\}$: Index for drugs.

**Parameters:**

*  $c_i$: Purchasing cost per unit of raw material $i$.
*  $o_j$: Operational cost per unit of drug $j$.
*  $s_j$: Selling price per unit of drug $j$.
*  $B$: Total budget for purchasing raw materials and operating the production process.
*  $K_j$: Production capacity for drug $j$ (in units).
*  $S_i$: Storage capacity for raw material $i$ (in units).
*  $R_{ij}$: Amount of raw material $i$ required to produce one unit of drug $j$.
*  $A_j$: Required amount of active ingredient in one unit of drug $j$.
*  $E_i$: Nominal amount of active ingredient extracted from one unit of raw material $i$.
*  $\alpha_1 = 0.005$: Variability of active ingredient extraction from raw material 1.
*  $\alpha_2 = 0.02$: Variability of active ingredient extraction from raw material 2.

**Decision Variables:**

*  $x_i$: Amount of r

In [282]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data
c = {1: 10, 2: 15}  # Cost of raw materials
o = {1: 5, 2: 8}  # Operational cost of drugs
s = {1: 30, 2: 45}  # Selling price of drugs
B = 1000  # Total budget
K = {1: 100, 2: 150}  # Production capacity for drugs
S = {1: 200, 2: 250}  # Storage capacity for raw materials
R = {(1, 1): 0.5, (1, 2): 0.8, (2, 1): 0.3, (2, 2): 0.6}  # Raw material requirement
A = {1: 0.2, 2: 0.3}  # Active ingredient requirement
E = {1: 0.4, 2: 0.7}  # Nominal amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Variability of active ingredient extraction

# Create a model
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material to purchase
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)  # Amount of drug to produce

# Objective function
model.profit = pyo.Obj